# Learning a causal model

The following example is adapted from "Elements of Causal Inference" by Jonas Peters, Dominik Janzig, and Bernhard Schölkopf (2017).
See also Jonas Peters great [4-part lecture series on causality](https://www.youtube.com/watch?v=zvrcyqcN9Wo).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoLars

In [ ]:
def sample_data(seed=None, n=10000, data_drift=False, concept_drift=False):
    # generate a sample from the distribution entailed by the causal graph
    np.random.seed(seed)
    # C, A, K
    C = np.random.randn(n)
    A = 0.8*np.random.randn(n)
    K = A + 0.1*np.random.randn(n)
    # X with and without data drift
    if data_drift:
        X = C - 2*A + 2.0*np.random.randn(n)
    else:
        X = C - 2*A + 0.2*np.random.randn(n)
    # F 
    F = 3*X + 0.8*np.random.randn(n)
    # D with and without concept drift
    if concept_drift:
        D =  2*X + 0.5*np.random.randn(n)
    else:
        D = -2*X + 0.5*np.random.randn(n)
    # G, Y 
    G = D + 0.5*np.random.randn(n)
    Y = 2*K - D + 0.2*np.random.randn(n)
    # H with and without data drift
    if data_drift:
        H = 0.5*Y + 1.0*np.random.randn(n)
    else:
        H = 0.5*Y + 0.1*np.random.randn(n)  
    # put all in a nice dataframe
    df = pd.DataFrame(np.vstack([C, A, K, X, F, D, G, Y, H]).T, columns=["C", "A", "K", "X", "F", "D", "G", "Y", "H"])
    return df

def test_model(input_vars, df_train, df_test):
    # fit model with given variables
    lm = LinearRegression().fit(df_train[input_vars], df_train[["Y"]])
    # check model fit and coefficients
    # true coefficients would be all values on the edges multiplied from the path from X to Y
    print(f"R^2 (train): {lm.score(df_train[input_vars], df_train[['Y']]):0.3f}; (test): {lm.score(df_test[input_vars], df_test[['Y']]):0.3f} => Y ~ {lm.intercept_[0]:0.3f} + " + " + ".join([f"{lm.coef_[0, i]:0.3f} * {input_vars[i]}" for i in range(len(input_vars))]))


## Train & evaluate the model

In [ ]:
# generate train and test data with different random seeds
df_train = sample_data(1)
df_test = sample_data(2)

In [ ]:
# (1) missing relevant input feature K -> wrong coefficient for X
test_model(["X"], df_train, df_test)

In [ ]:
# (2) all the right input features -> correct coefficients
test_model(["X", "K"], df_train, df_test)

In [ ]:
# (3) additional input feature D, which has a more direct influence on Y than X
test_model(["X", "K", "D"], df_train, df_test)

In [ ]:
# (4) additional input feature H, which is dependent on (i.e. highly correlated with) Y
test_model(["X", "K", "H"], df_train, df_test)

In [ ]:
# (5) regularized model with all inputs -> chooses dependent variable H instead of causal influences
input_vars = [c for c in df_train.columns if c != "Y"]
lm = LassoLars(alpha=0.003).fit(df_train[input_vars], df_train["Y"])
print(f"R^2 (train): {lm.score(df_train[input_vars], df_train['Y']):0.3f}; (test): {lm.score(df_test[input_vars], df_test['Y']):0.3f} => Y ~ {lm.intercept_:0.3f} + " + " + ".join([f"{lm.coef_[i]:0.3f} * {input_vars[i]}" for i in range(len(input_vars))]))

## Data & Concept Drifts

In [ ]:
# generate test data with a data drift in X and H
df_test = sample_data(2, data_drift=True)

In [ ]:
# model (2): true relationship between X and Y -> test performance equally good
test_model(["X", "K"], df_train, df_test)

In [ ]:
# model (4): variable dependent on but not causal of Y -> test performance a lot worse
test_model(["X", "K", "H"], df_train, df_test)

In [ ]:
# generate test data with a concept drift in D, i.e., on the way from X to Y
df_test = sample_data(2, concept_drift=True)

In [ ]:
# model (2): causal relationship between X and Y changed -> test performance catastrophic
test_model(["X", "K"], df_train, df_test)